In [160]:
import pandas as pd
from lxml import etree
import requests

---- using HTML response ----

In [161]:
url = "https://www.hemnet.se/bostader"
params= {
    'housing_form_groups':'apartments',
    'location_ids':'473498',
    'item_types':'bostadsratt'
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
response.url

'https://www.hemnet.se/bostader?housing_form_groups=apartments&location_ids=473498&item_types=bostadsratt'

--------- w beautiful soup ------------

In [162]:
from bs4 import BeautifulSoup

In [163]:
soup = BeautifulSoup(response.content, "html.parser")

---- search thru all the cards in the ul list ---

In [167]:
cards=soup.select('#result > ul > li > a')
#cards

In [168]:
#example output
li_results=soup.select('#result > ul > li')
li_results[0].attrs['data-gtm-item-info']

'{"id":"19124215","name":"Armégatan 3,  Högt upp i huset!","active_toplisting":true,"toplisting":{"id":113084,"active":true},"type":"raketen,premium","has_highlighted_showings":true,"product_category":"premium/raketen"}'

---- following the item link and then get the json and id from one of the script tags ---

In [169]:
#first link to request - could be a loop for all items
item_url=cards[1].attrs['href']

item_response=requests.request("GET", item_url, headers=headers)
item_soup=BeautifulSoup(item_response.content, "html.parser")

item_response.url

'https://www.hemnet.se/bostad/lagenhet-3rum-solna-kommun-armegatan-11b-19197055'

In [170]:
import json

#but... since the site uses dynamic loading of the pictures we can't get much data from the html here. Using script tags with jsons works however

#not working for example
#ilist=item_soup.find_all(attrs={"data-testid": "gallery-right-buttons"})

ilist=item_soup.find_all("script",type="application/ld+json")

#since nr of item changes we need to search all of them thru to find the right one, skip the first since that is badly formatted
for item in ilist[1:]:
    #print (item.text)
    jt=json.loads(item.text)
    if jt['@type']=='Product':
        #finally our product id
        print(jt['mpn'])

19197055


now we can either call the link to get stuff from that or call the graphql with the id